In [ ]:
import os
os.chdir("..")

In [ ]:
import h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sb
from scipy.stats import beta as BetaDistr
from TravelAndMutate.datamanager import getHeteroAttributes

In [ ]:
file = h5py.File("analysis/newfitness.h5")

allfitness = {name : pd.DataFrame.from_records(group["fitness_evolution"][:]) for name,group in file.items()}
fitness_params = getHeteroAttributes(file).sort_values(["fitness_p","fitness_delta+","fitness_delta-"])

file.close()

In [ ]:
nrows = int(np.ceil(len(allfitness) / 2))
fig, axs = plt.subplots(nrows, 2, figsize=(8,4*nrows))
axs = axs.flatten()
for i in range(fitness_params.shape[0]):
	name = str(fitness_params.index[i])
	sb.histplot(data=allfitness[name], x="t", y="phi", bins=50, ax=axs[i], cbar_kws={"norm":matplotlib.colors.LogNorm()})
	label = [f"Fitness p = {fitness_params.iloc[i]['fitness_p']}", r"Fitness $\delta^-$ "+f"= {fitness_params.iloc[i]['fitness_delta-']}", r"Fitness $\delta^+$ "+f"= {fitness_params.iloc[i]['fitness_delta+']}"]
	axs[i].text(0.05, 0.8, "\n".join(label), transform=axs[i].transAxes)
	axs[i].set_ylabel("Fitness")
	axs[i].set_xlabel("t")
	axs[i].set_xlim(left=0)
fig.tight_layout()
fig.savefig("analysis/fitness_evolution.png")